In [ ]:
# import env libraries
import os
from dotenv import load_dotenv
# import chatbot libraries
from langchain_openai import ChatOpenAI, OpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.prompts import ChatPromptTemplate, PromptTemplate
#import pdf embedding libraries
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader
# import pinecone
import pinecone
from langchain.vectorstores import Pinecone
import time

load_dotenv()

def initialize_pinecone(index_name):
    pinecone.init(
        api_key=os.environ.get('PINECONE_API_KEY'),
        environment=os.environ.get('PINECONE_ENVIRONMENT')
    )

    # create index if it does not exist
    if index_name not in pinecone.list_indexes():
        pinecone.create_index(
            index_name,
            dimension=1536,
            metric='cosine'
        )
    # wait for index to finish initialization
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

def initialize_chatbot(index_name):
    chatbot = ChatOpenAI(
        openai_api_key=os.environ["OPENAI_API_KEY"],
        model='gpt-3.5-turbo',
        temperature=0.2
    )

    embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])
    vectorstore = Pinecone.from_existing_index(index_name, embeddings)
    retriever = vectorstore.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={'score_threshold': 0.9}
    )

    memory = ConversationBufferWindowMemory(
        k=10,
        memory_key="chat_history",
        return_messages=True,
        output_key='answer',
    )

    qa = ConversationalRetrievalChain.from_llm(
        llm=chatbot,
        retriever=retriever,
        memory=memory,
        return_source_documents=True,
        rephrase_question=False,
        response_if_no_docs_found="Sorry, I have no information about that.",
        verbose=True
    )

    return qa

def embed_pdfs(file_path):
    loader = PyMuPDFLoader(file_path)
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
    all_splits = text_splitter.split_documents(data)

    embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])
    Pinecone.from_documents(documents=all_splits, embedding=embeddings())

def chat(query, qa):
    response = qa.invoke(query)

    print(response["answer"])
    print('\n')
    print(response)

In [ ]:
index_name = "recipes"

initialize_pinecone(index_name)
chatbot = initialize_chatbot(index_name)

In [ ]:
query = "what is the recipe for boiled eggs?"

chat(query, chatbot)

In [ ]:
query = "give me the nutritional information?"

chat(query, chatbot)

In [ ]:
query = "what is the length of the space station"

chat(query, chatbot)

In [ ]:
memory = ConversationBufferWindowMemory()
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "im good how are you"}, {"output": "im good too"})
memory.load_memory_variables({})





In [ ]:
llm = ChatOpenAI(
        openai_api_key=os.environ["OPENAI_API_KEY"],
        model='gpt-3.5-turbo',
        temperature=0
    )
template = (
    "Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question."
    "\n\nChat History: {chat_history}"
    "\n\nStandalone question: {question}"
)
prompt = ChatPromptTemplate.from_template(template)
memory = ConversationBufferWindowMemory(
        k=10,
        memory_key="chat_history",
        return_messages=True
    )
memory.save_context({"input": "how tall is john"}, {"output": "6 ft"})

chain = LLMChain(llm=llm, prompt=prompt, memory=memory)
chain.invoke("how fast is he")

template = (
    "System: Use the following pieces of context to answer the user's question." 
    "If you don't know the answer, just say that you don't know, don't try to make up an answer."
    "\n----------------"
    "\n{docs}"
    "\n----------------"
    "\nHuman: {question}"
)



In [ ]:
memory.load_memory_variables({})